In [27]:
#importer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from math import sqrt


# Lese inn data og basic cleaning

In [34]:
# Last inn datasettet
data = pd.read_csv('filer/train.csv', low_memory=False)

# Sjekk datatyper og om det er noen manglende verdier
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1017209 entries, 0 to 1017208
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   Store          1017209 non-null  int64 
 1   DayOfWeek      1017209 non-null  int64 
 2   Date           1017209 non-null  object
 3   Sales          1017209 non-null  int64 
 4   Customers      1017209 non-null  int64 
 5   Open           1017209 non-null  int64 
 6   Promo          1017209 non-null  int64 
 7   StateHoliday   1017209 non-null  object
 8   SchoolHoliday  1017209 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 69.8+ MB


In [18]:
# Konverter 'Date'-kolonnen til datetime-typen
data['Date'] = pd.to_datetime(data['Date'])

# Sjekk unike verdier i 'StateHoliday'-kolonnen
data['StateHoliday'].unique()

# Kartlegg 'StateHoliday'-kolonnen til heltall
state_holiday_mapping = {'0': 0, 'a': 1, 'b': 2, 'c': 3}
data['StateHoliday'] = data['StateHoliday'].map(state_holiday_mapping)

# Sjekk de første radene i dataene for å bekrefte endringene
data.head()


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


# Random forest regressor for å forecaste salg

#### steg 1: lage og trene modell

In [19]:
# Trekk ut år og måned fra 'Date'-kolonnen som nye funksjoner
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month

# Definer funksjonene og målvariabelen
features = ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'Year', 'Month']
target = 'Sales'

X = data[features]
y = data[target]


#elage modellen
model = RandomForestRegressor(n_estimators=100, random_state=42)

#Trene modellen
model.fit(X, y)


RandomForestRegressor(random_state=42)

#### Steg 2: Teste model på test data

In [22]:
# Last inn det nye datasettet
test_data = pd.read_csv('filer/test.csv')

# Fyll manglende verdier i 'Open'-kolonnen med 0 (indikerer at butikken var stengt)
test_data['Open'].fillna(0, inplace=True)

# Utfør de samme forbehandlingsstegene som treningsdataen
test_data['Date'] = pd.to_datetime(test_data['Date'])
test_data['Year'] = test_data['Date'].dt.year
test_data['Month'] = test_data['Date'].dt.month
test_data['StateHoliday'] = test_data['StateHoliday'].map(state_holiday_mapping)

# Velg de samme funksjonene som treningsdataen
X_test = test_data[features]

# Bruk den trente modellen til å lage prediksjoner
test_data['Sales'] = model.predict(X_test)



In [24]:
# Or print specific columns
print(test_data[['Store', 'Date', 'Sales']])

       Store       Date         Sales
0          1 2015-09-17   4029.118167
1          3 2015-09-17   7596.922000
2          7 2015-09-17   9489.738000
3          8 2015-09-17   6939.720357
4          9 2015-09-17   6718.573000
...      ...        ...           ...
41083   1111 2015-08-01   2566.175206
41084   1112 2015-08-01   6984.704512
41085   1113 2015-08-01   5681.542835
41086   1114 2015-08-01  21386.945745
41087   1115 2015-08-01   8118.902440

[41088 rows x 3 columns]


#### Steg 3: Evaluere modell

In [28]:
# Del det opprinnelige treningsdataet i et nytt treningssett (første 80%) og et valideringssett (siste 20%)
treningsdata = data.iloc[:int(0.8*len(data))]
valideringsdata = data.iloc[int(0.8*len(data)):]

# Tren modellen på det nye treningssettet
X_trening = treningsdata[features]
y_trening = treningsdata[target]
model.fit(X_trening, y_trening)

# Gjør prediksjoner på valideringssettet
X_val = valideringsdata[features]
y_val = valideringsdata[target]
y_pred = model.predict(X_val)

# Beregn evalueringsmetrikker
mae = mean_absolute_error(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
rmse = sqrt(mse)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Root Mean Squared Error (RMSE): {rmse}")


Mean Absolute Error (MAE): 711.0744898436153
Mean Squared Error (MSE): 1353003.4499209486
Root Mean Squared Error (RMSE): 1163.186764849458


In [32]:
#finn salg gitt gjennomsnittlig salg
mean_sales = y_trening.mean()
y_pred_mean = [mean_sales] * len(y_val)

#Beregn evalueringsmetrics
mae_mean = mean_absolute_error(y_val, y_pred_mean)
mse_mean = mean_squared_error(y_val, y_pred_mean)
rmse_mean = sqrt(mse_mean)

print(f"Mean Absolute Error of Mean Model (MAE): {mae_mean}")
print(f"Mean Squared Error of Mean Model (MSE): {mse_mean}")
print(f"Root Mean Squared Error of Mean Model (RMSE): {rmse_mean}")

Mean Absolute Error of Mean Model (MAE): 2831.3907955853033
Mean Squared Error of Mean Model (MSE): 14086237.245521732
Root Mean Squared Error of Mean Model (RMSE): 3753.163631594249


#### Steg 4: fylle ut csv fil med forecast

In [29]:
# Lag en ny DataFrame med 'Id' og predikerte 'Sales'
innsending = pd.DataFrame({
    'Id': test_data['Id'],
    'Sales': test_data['Sales']  # antar at 'Sales' inneholder de predikerte verdiene
})

# Skriv DataFrame til en CSV-fil
innsending.to_csv('filer/innsending.csv', index=False)
